In [9]:
"""
Author: Aditya Jain
Date last modified  : August 24, 2023
About : This script predicts the class given an input image
"""
import sys, os
sys.path.append('/home/mila/a/aditya.jain/mothAI/gbif_species_trainer/model_training/')

from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
from models.resnet50 import Resnet50
import json
import numpy as np

image_path   = 'test-moth.png'
image_resize = 224
config_file  = '/home/mila/a/aditya.jain/mothAI/gbif_species_trainer/model_training/config/01-config_quebec-vermont.json' 
model_path   = '/home/mila/a/aditya.jain/logs/quebec-vermont-moth-model_v07_resnet50_2022-12-22-07-54.pt'
label_file   = '/home/mila/a/aditya.jain/mothAI/gbif_species_trainer/model_training/data/quebec-vermont_numeric_labels.json'

In [11]:
# model loading
f           = open(config_file)
config_data = json.load(f)

device      = "cuda" if torch.cuda.is_available() else "cpu"
model       = Resnet50(config_data).to(device)

checkpoint  = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of SymInts size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [12]:
# image loading and processing
image     = Image.open(image_path)

transform = transforms.Compose([
                        transforms.Resize((image_resize, image_resize)),              # resize the image to 224x224 
                        transforms.ToTensor()])
image     = transform(image)
image     = torch.unsqueeze(image, 0).to(device)

In [21]:
# prediction
prediction      = model(image)
_, predict_indx = torch.topk(prediction, 1)

In [38]:
f            = open(label_file)
label_info   = json.load(f)
species_list = label_info['species_list']

print('The predicted species is : ', species_list[predict_indx])

The predicted species is :  Sphinx luscitiosa


In [48]:
# finding top 3 predictions
_, predict_indx = torch.topk(prediction, 3)
predict_indx    = predict_indx[0]

for index in predict_indx:
    print(species_list[index])

Sphinx luscitiosa
Hemileuca lucina
Proserpinus flavofasciata


In [12]:
from PIL import Image

image_path   = 'test-moth.png'


In [14]:
image     = Image.open(image_path)